In [ ]:
# Imports as always.
import os
import re
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

from PIL import Image

from hflayers import Hopfield, HopfieldLayer, HopfieldPooling

from datetime import datetime

# Get the current data and time as a string.
date_string = datetime.now().strftime('%Y-%m-%d-(%H-%M-%S)')

# Ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(f'CUDA is available for use with PyTorch: {torch.cuda.is_available()}')

In [ ]:
import sys

print(f'Installed Python version:  {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')
print(f'Installed PyTorch version: {torch.__version__}')

In [ ]:
# Change this as necessary. Unzip all downloaded data into this directory, keeping default naming.
data_directory = './data/ISIC2018/'

In [ ]:
# Helper function to send a tensor/model/etc. to the CPU/GPU accordingly.
def to_device(x):
    if torch.cuda.is_available():
        return x.cuda()
    else:
        return x.cpu()

# Test-time Reconstruction for Lesion Segmentation

"Test-time" reconstruction refers to a task of our own proposing consisting of the training of a model on 'perfect' data, then testing its performance on 'damaged' data. The goal here is to develop a model that has some innate understanding of the training data's form so that it is able to 'reconstruct' the perfection from the damage at test-time, and thus succeed at the task (e.g. segmentation) irrespective of the damage. *Importantly, we do not use damaged data for a validation set*, as the model should not be exposed to any form of damage prior to testing. 

Keeping to a consistent theme, we are considering the segmentation of lesions in the ISIC 2018 dataset.

We are keen to explore the ability for a continuous Hopfield network to enable good test-time reconstructive capabilities, as an immediate by-product of the intended infusion of memory. To do this, the following experiments are conducted:
- **End-to-end auto-reconstruction**: optimistically, one hopes that simply using Hopfield layers within the architecture is enough to learn this reconstructive ability automatically. This experiment looks to train a memory-infused model end-to-end for lesion segmentation, then test on damaged data to observe the natural ability for test-time reconstruction without specific prompting or pre-training.
- **Pre-trained reconstructive unit(s)**: pessimistically, one expects that a lack of reconstructive training leaves the Hopfield layers without a specific goal and so without a natural ability. This experiment pre-trains one or more Hopfield layers to reconstruct lesions and/or target masks, then employs them as a pre-processing unit for an otherwise standard lesion segmentation model.
- **Post-process reconstruction**: another idea is to consider Hopfield layers as being able to correct output from kernel-style models to reconstruct the model's process itself, rather than the data. This experiment exclusively uses Hopfield layers (and Hopfield pooling layers) following convolution layers so that, at test-time, the kernel activation provides input to the Hopfield layer to be repaired following exposure to damaged data.

In all experiments, the test masks will be perfect. If this were not the case, we would be evaluating the performance of our model to predict masks damaged in the same way as the data! And while this would be very interesting -- that the model has learned the pattern of damage -- it does not help us in defining a reconstructive model outright.

A further experiment we might like to perform is to explore how capable these Hopfield layers are in defining models capable of *measuring the degree of damage* in its reconstruction pipeline. Simplistically, we might take the energy difference between the damaged input and perfect output.

A final point: it is worth considering whether training a model to have test-time reconstructive function limits or degrades its task performance on perfect data. We should explore whether a model showing good reconstructive performance exhibits a degradation to segmentation performance when tested on pure data. If such degradation exists, then it may become desirable to have an independent, pre-trained reconstructive unit available as a pre-processing step that one may opt not to use if the data is understood to be of high quality. In practice, you can imagine that we might only apply our pre-processing reconstruction if the data are very likely to be damaged somehow (e.g. receiving medical imagery over a compromised communication channel, working with uncooperative patients that may hinder data collection methods, etc.) 